# NBA Contract Value Project #

### Importing Tools ###

To carry out the project, I imported applicable python libraries and some from UC Berkeley's depositories, such as the datascience library. The following cell carries out the importing of libraries used in this project:

Note: There is a possibility that the following cell won't run because the datascience library is only accessible to those with UC Berkeley credentials. 

In [ ]:
import numpy as np
import pandas as pd
from datascience import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from scipy.stats import zscore

### Loading Datasets ###

The following cells import the datasets from basketballreference.com: 

In [431]:
contracts = Table.read_table('2018_2019_Contracts.csv', sep=',')
contracts.show(10)

# importing the contracts dataset

Rk,Player,Tm,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,Signed Using,Guaranteed
1,Stephen Curry\curryst01,GSW,$37457154,$40231758,$43006362,$45780966,nan,nan,Bird Rights,$166476240
2,Chris Paul\paulch01,HOU,$35654150,$38506482,$41358814,$44211146,nan,nan,nan,$159730592
3,Russell Westbrook\westbru01,OKC,$35654150,$38178000,$41006000,$43848000,$46662000,nan,Bird Rights,$158686150
4,LeBron James\jamesle01,LAL,$35654150,$37436858,$39219565,$41002273,nan,nan,nan,$113310573
5,Blake Griffin\griffbl01,DET,$32088932,$34234964,$36595996,$38957028,nan,nan,Bird Rights,$102919892
6,Gordon Hayward\haywago01,BOS,$31214295,$32700690,$34187085,nan,nan,nan,Cap space,$63914985
7,Kyle Lowry\lowryky01,TOR,$31200000,$33296296,nan,nan,nan,nan,Bird Rights,$64496296
8,Paul George\georgpa01,OKC,$30560700,$33005556,$35450412,$37895268,nan,nan,Maximum Salary,$99016668
9,Mike Conley\conlemi01,MEM,$30521116,$32511623,$34504132,nan,nan,nan,Cap Space,$85459122
10,James Harden\hardeja01,HOU,$30431854,$37800000,$40824000,$43848000,$46872000,nan,Bird Rights,$152903854


In [432]:
boxscore_clean = Table.read_table('2017_2018_seasonclean.csv', sep=',')
boxscore_clean.show(10)

#importing the players' 2017-2018 season performance

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G
1,Alex Abrines\abrinal01,SG,24,OKC,75,8,15.1,1.5,3.9,0.395,1.1,2.9,0.38,0.4,0.9,0.443,0.54,0.5,0.6,0.848,0.3,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7
2,Quincy Acy\acyqu01,PF,27,BRK,70,8,19.4,1.9,5.2,0.356,1.5,4.2,0.349,0.4,1,0.384,0.496,0.7,0.9,0.817,0.6,3.1,3.7,0.8,0.5,0.4,0.9,2.1,5.9
3,Steven Adams\adamsst01,C,24,OKC,76,76,32.7,5.9,9.4,0.629,0,0,0,5.9,9.3,0.631,0.629,2.1,3.8,0.559,5.1,4,9,1.2,1.2,1,1.7,2.8,13.9
4,Bam Adebayo\adebaba01,C,20,MIA,69,19,19.8,2.5,4.9,0.512,0,0.1,0,2.5,4.8,0.523,0.512,1.9,2.6,0.721,1.7,3.8,5.5,1.5,0.5,0.6,1,2,6.9
5,Arron Afflalo\afflaar01,SG,32,ORL,53,3,12.9,1.2,3.1,0.401,0.5,1.3,0.386,0.7,1.7,0.413,0.485,0.4,0.5,0.846,0.1,1.2,1.2,0.6,0.1,0.2,0.4,1.1,3.4
6,Cole Aldrich\aldrico01,C,29,MIN,21,0,2.3,0.2,0.7,0.333,0,0,nan,0.2,0.7,0.333,0.333,0.1,0.3,0.333,0.1,0.6,0.7,0.1,0.1,0,0,0.5,0.6
7,LaMarcus Aldridge\aldrila01,C,32,SAS,75,75,33.5,9.2,18,0.51,0.4,1.2,0.293,8.8,16.7,0.526,0.52,4.5,5.3,0.837,3.3,5.2,8.5,2,0.6,1.2,1.5,2.1,23.1
8,Jarrett Allen\allenja01,C,19,BRK,72,31,20,3.3,5.5,0.589,0.1,0.2,0.333,3.2,5.3,0.599,0.596,1.6,2,0.776,2,3.4,5.4,0.7,0.4,1.2,1.1,2,8.2
9,Kadeem Allen\allenka01,PG,25,BOS,18,1,5.9,0.3,1.2,0.273,0,0.6,0,0.3,0.6,0.545,0.273,0.4,0.5,0.778,0.2,0.4,0.6,0.7,0.2,0.1,0.5,0.8,1.1
10,Tony Allen\allento01,SF,36,NOP,22,0,12.4,2,4.1,0.484,0.2,0.5,0.333,1.8,3.6,0.506,0.505,0.5,1,0.524,0.9,1.2,2.1,0.4,0.5,0.1,0.9,2.2,4.7


In [433]:
rookies_17 = Table.read_table('rookies_2017-18.csv', sep=',')
rookies_17.show(10)

Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Totals,Totals.1,Totals.2,Totals.3,Totals.4,Totals.5,Totals.6,Totals.7,Totals.8,Totals.9,Totals.10,Totals.11,Totals.12,Totals.13,Unnamed: 21,Unnamed: 22,Unnamed: 23,Per Game,Per Game.1,Per Game.2,Per Game.3
Rk,Player,Debut,Age,Yrs,G,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P,FT%,MP,PS/G,TRB,AST
1,Bam Adebayo\adebaba01,01/10/18 MIA @ IND,20,2,89,1742,228,441,0,10,178,249,159,519,125,45,56,91,190,634,.517,.000,.715,19.6,7.1,5.8,1.4
2,Jarrett Allen\allenja01,01/1/18 BRK vs. ORL,19,2,91,1962,326,554,7,30,164,221,200,547,78,39,119,103,197,823,.588,.233,.742,21.6,9.0,6.0,0.9
3,Kadeem Allen\allenka01,01/31/18 BOS vs. NYK,25,1,18,107,6,22,0,11,7,9,4,11,12,3,2,9,15,19,.273,.000,.778,5.9,1.1,0.6,0.7
4,Ike Anigbogu\anigbik01,01/14/18 IND @ PHO,19,2,14,36,4,12,0,0,5,6,6,12,1,1,4,3,1,13,.333,nan,.833,2.6,0.9,0.9,0.1
5,OG Anunoby\anunoog01,01/1/18 TOR vs. MIL,20,2,90,1830,222,471,93,260,43,75,54,222,64,63,19,55,170,580,.471,.358,.573,20.3,6.4,2.5,0.7
6,Ryan Arcidiacono\arcidry01,01/20/18 CHI @ ATL,23,2,44,870,70,154,44,110,15,20,4,77,119,30,1,30,70,199,.455,.400,.750,19.8,4.5,1.8,2.7
7,Jamel Artis\artisja01,03/22/18 ORL vs. PHI,25,1,15,279,31,79,8,29,7,12,4,38,18,2,3,8,11,77,.392,.276,.583,18.6,5.1,2.5,1.2
8,Dwayne Bacon\bacondw01,01/17/18 CHO vs. WAS,22,2,65,883,104,256,20,61,30,37,6,149,48,19,2,30,66,258,.406,.328,.811,13.6,4.0,2.3,0.7
9,Lonzo Ball\balllo01,01/11/18 LAL vs. SAS,20,2,72,2312,267,721,118,375,40,85,87,471,466,108,47,171,159,692,.370,.315,.471,32.1,9.6,6.5,6.5


### Data Processing ###

The following few cells process the datasets seen above. These calls clean the dataset by removing duplicate players found in the tables. Duplicate rows for the same player are created when the player is traded mid-season. The total averages for the player during the season are kept in the table, so that player can still be included in our model. Furthermore, for the contract value model, it is important to use players who are not under rookie contracts, as most rookies are underpaid for how well some play. 

In [434]:
joined_17 = boxscore_clean.join('Player', rookies_17, 'Unnamed: 1')

In [435]:
rookies_names_17 = joined_17.column(0)

In [436]:
rookies_names_17.item(len(rookies_names_17)-1)

'Zhou Qi\\qizh01'

In [312]:
for name in np.arange(len(rookies_names_17)-1):
    boxscore_clean = boxscore_clean.where('Player', are.not_equal_to(rookies_names_17.item(name)))
    
#cut out 2017 rookies

In [437]:
rookies_16 = Table.read_table('rookies_2016-17.csv', sep=',')
joined_16 = boxscore_clean.join('Player', rookies_16, 'Player')
rookies_names_16 = joined_16.column(0)
for name in np.arange(len(rookies_names_16)-1):
    boxscore_clean = boxscore_clean.where('Player', are.not_equal_to(rookies_names_16.item(name)))
#cut out 2016 rookies

In [438]:
rookies_15 = Table.read_table('rookies_2015-16.csv', sep=',')
joined_15 = boxscore_clean.join('Player', rookies_15, 'Player')
rookies_names_15 = joined_15.column(0)
for name in np.arange(len(rookies_names_15)-1):
    boxscore_clean = boxscore_clean.where('Player', are.not_equal_to(rookies_names_15.item(name)))
#cut out 2015 rookies

### Cleaned Player and Contracts Data Table ###

The following cells displays the cleaned datasets used to generate the contract value model:

In [439]:
boxscore_clean

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G
2,Quincy Acy\acyqu01,PF,27,BRK,70,8,19.4,1.9,5.2,0.356,1.5,4.2,0.349,0.4,1,0.384,0.496,0.7,0.9,0.817,0.6,3.1,3.7,0.8,0.5,0.4,0.9,2.1,5.9
3,Steven Adams\adamsst01,C,24,OKC,76,76,32.7,5.9,9.4,0.629,0,0,0,5.9,9.3,0.631,0.629,2.1,3.8,0.559,5.1,4,9,1.2,1.2,1,1.7,2.8,13.9
4,Bam Adebayo\adebaba01,C,20,MIA,69,19,19.8,2.5,4.9,0.512,0,0.1,0,2.5,4.8,0.523,0.512,1.9,2.6,0.721,1.7,3.8,5.5,1.5,0.5,0.6,1,2,6.9
5,Arron Afflalo\afflaar01,SG,32,ORL,53,3,12.9,1.2,3.1,0.401,0.5,1.3,0.386,0.7,1.7,0.413,0.485,0.4,0.5,0.846,0.1,1.2,1.2,0.6,0.1,0.2,0.4,1.1,3.4
6,Cole Aldrich\aldrico01,C,29,MIN,21,0,2.3,0.2,0.7,0.333,0,0,nan,0.2,0.7,0.333,0.333,0.1,0.3,0.333,0.1,0.6,0.7,0.1,0.1,0,0,0.5,0.6
7,LaMarcus Aldridge\aldrila01,C,32,SAS,75,75,33.5,9.2,18,0.51,0.4,1.2,0.293,8.8,16.7,0.526,0.52,4.5,5.3,0.837,3.3,5.2,8.5,2,0.6,1.2,1.5,2.1,23.1
8,Jarrett Allen\allenja01,C,19,BRK,72,31,20,3.3,5.5,0.589,0.1,0.2,0.333,3.2,5.3,0.599,0.596,1.6,2,0.776,2,3.4,5.4,0.7,0.4,1.2,1.1,2,8.2
9,Kadeem Allen\allenka01,PG,25,BOS,18,1,5.9,0.3,1.2,0.273,0,0.6,0,0.3,0.6,0.545,0.273,0.4,0.5,0.778,0.2,0.4,0.6,0.7,0.2,0.1,0.5,0.8,1.1
10,Tony Allen\allento01,SF,36,NOP,22,0,12.4,2,4.1,0.484,0.2,0.5,0.333,1.8,3.6,0.506,0.505,0.5,1,0.524,0.9,1.2,2.1,0.4,0.5,0.1,0.9,2.2,4.7
11,Al-Farouq Aminu\aminual01,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3


In [440]:
contracts

Rk,Player,Tm,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,Signed Using,Guaranteed
1,Stephen Curry\curryst01,GSW,$37457154,$40231758,$43006362,$45780966,nan,nan,Bird Rights,$166476240
2,Chris Paul\paulch01,HOU,$35654150,$38506482,$41358814,$44211146,nan,nan,nan,$159730592
3,Russell Westbrook\westbru01,OKC,$35654150,$38178000,$41006000,$43848000,$46662000,nan,Bird Rights,$158686150
4,LeBron James\jamesle01,LAL,$35654150,$37436858,$39219565,$41002273,nan,nan,nan,$113310573
5,Blake Griffin\griffbl01,DET,$32088932,$34234964,$36595996,$38957028,nan,nan,Bird Rights,$102919892
6,Gordon Hayward\haywago01,BOS,$31214295,$32700690,$34187085,nan,nan,nan,Cap space,$63914985
7,Kyle Lowry\lowryky01,TOR,$31200000,$33296296,nan,nan,nan,nan,Bird Rights,$64496296
8,Paul George\georgpa01,OKC,$30560700,$33005556,$35450412,$37895268,nan,nan,Maximum Salary,$99016668
9,Mike Conley\conlemi01,MEM,$30521116,$32511623,$34504132,nan,nan,nan,Cap Space,$85459122
10,James Harden\hardeja01,HOU,$30431854,$37800000,$40824000,$43848000,$46872000,nan,Bird Rights,$152903854


In [441]:
full_clean_table = boxscore_clean.join('Player', contracts, 'Player')

In [444]:
full_clean_table.show(10)

#joining the two dataset together

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,Signed Using,Guaranteed
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,$21590909,$19863636,$18136364,$16409091,nan,nan,nan,$76000000
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,$1378242,$1618520,$1752950,nan,nan,nan,nan,$1378242
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,$28928710,$30123015,nan,nan,nan,nan,Cap Space,$28928710
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,$4000000,nan,nan,nan,nan,nan,nan,$4000000
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,$6957105,nan,nan,nan,nan,nan,Cap Space,$6957105
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,$11536515,nan,nan,nan,nan,nan,1st Round Pick,$11536515
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,$4350000,$4160000,nan,nan,nan,nan,Room Exception,$8510000
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,$1349383,nan,nan,nan,nan,nan,nan,nan
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,$1349383,nan,nan,nan,nan,nan,nan,nan
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,$18500000,$18500000,nan,nan,nan,nan,Cap Space,$18500000


In [445]:
clean_data = full_clean_table.drop('2019-20').drop('2020-21').drop('2021-22').drop('2022-23').drop('2023-24').drop('Signed Using').drop('Guaranteed')

# cares only about 2018-2019 contract

In [447]:
clean_data

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,$21590909
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,$1378242
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,$28928710
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,$4000000
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,$6957105
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,$11536515
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,$4350000
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,$1349383
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,$1349383
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,$18500000


Basic type conversions are carried out to make use of the data without certain characters hindering the calculations.

In [448]:
def convert_string_to_number(string):
    return float(string.strip('$'))*1

In [449]:
convert_string_to_number('$21590909')

21590909.0

In [451]:
clean_data.show(10)
#convert strings to number

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,$21590909
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,$1378242
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,$28928710
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,$4000000
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,$6957105
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,$11536515
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,$4350000
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,$1349383
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,$1349383
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,$18500000


## Final Data Tables ##

The following cells display the fully processed data tables:

In [452]:
clean_data['2018-19'] = clean_data.apply(convert_string_to_number, '2018-19')

In [453]:
clean_data

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,2.15909e+07
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,1.37824e+06
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,2.89287e+07
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,4e+06
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,6.95710e+06
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,1.15365e+07
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,4.35e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,1.34938e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,1.34938e+06
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,1.85e+07


In [454]:
clean_data

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,2.15909e+07
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,1.37824e+06
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,2.89287e+07
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,4e+06
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,6.95710e+06
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,1.15365e+07
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,4.35e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,1.34938e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,1.34938e+06
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,1.85e+07


In [455]:
players = clean_data.where('Player', are.not_equal_to('nan'))

In [456]:
players

#drop all the NAN

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,2.15909e+07
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,1.37824e+06
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,2.89287e+07
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,4e+06
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,6.95710e+06
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,1.15365e+07
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,4.35e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,1.34938e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,1.34938e+06
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,1.85e+07


## Data Table Processing ##

In [458]:
players

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,2.15909e+07
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,1.37824e+06
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,2.89287e+07
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,4e+06
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,6.95710e+06
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,1.15365e+07
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,4.35e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,1.34938e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,1.34938e+06
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,1.85e+07


## Separate table by Position: ##

In [459]:
#PG
point_guard = players.where('Pos', are.equal_to('PG'))
point_guard

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Ben Simmons\simmobe01,450,PG,21,PHI,81,81,33.7,6.7,12.3,0.545,0,0.1,0,6.7,12.2,0.551,0.545,2.4,4.2,0.56,1.8,6.3,8.1,8.2,1.7,0.9,3.4,2.6,15.8,184,PHI,6.43452e+06
Chris Paul\paulch01,396,PG,32,HOU,58,58,31.8,6.3,13.8,0.46,2.5,6.5,0.38,3.8,7.2,0.532,0.55,3.5,3.8,0.919,0.6,4.8,5.4,7.9,1.7,0.2,2.2,2.4,18.6,2,HOU,3.56542e+07
Cory Joseph\josepco01,269,PG,26,IND,82,17,27,3.1,7.4,0.424,0.8,2.3,0.353,2.3,5,0.457,0.479,0.9,1.1,0.745,0.5,2.7,3.2,3.2,1,0.2,1.1,1.7,7.9,161,IND,7.945e+06
D.J. Augustin\augusdj01,24,PG,30,ORL,75,36,23.5,3.3,7.2,0.452,1.5,3.6,0.419,1.7,3.6,0.485,0.557,2.2,2.5,0.868,0.4,1.7,2.1,3.8,0.7,0,1.6,1.3,10.2,171,ORL,7.25e+06
Damian Lillard\lillada01,298,PG,27,POR,73,73,36.6,8.5,19.4,0.439,3.1,8.6,0.361,5.4,10.8,0.501,0.519,6.8,7.4,0.916,0.8,3.6,4.5,6.6,1.1,0.4,2.8,1.6,26.9,14,POR,2.79777e+07
Dante Exum\exumda01,153,PG,22,UTA,14,0,16.8,3,6.2,0.483,0.4,1.3,0.278,2.6,4.9,0.536,0.511,1.8,2.2,0.806,0.3,1.6,1.9,3.1,0.6,0.2,1.4,1.4,8.1,138,UTA,9.6e+06
Darren Collison\collida01,104,PG,30,IND,69,64,29.2,4.6,9.2,0.495,1.4,3,0.468,3.2,6.3,0.508,0.571,1.8,2.1,0.882,0.5,2,2.6,5.3,1.3,0.2,1.2,1.8,12.4,135,IND,1e+07
De'Aaron Fox\foxde01,164,PG,20,SAC,73,60,27.8,4.5,10.9,0.412,0.6,2.1,0.307,3.8,8.8,0.436,0.441,1.9,2.7,0.723,0.5,2.3,2.8,4.4,1,0.3,2.4,2.2,11.6,198,SAC,5.47092e+06
Dennis Schroder\schrode01,442,PG,24,ATL,67,67,31,7.5,17.1,0.436,1.1,3.9,0.29,6.3,13.2,0.48,0.469,3.4,4,0.849,0.7,2.4,3.1,6.2,1.1,0.1,2.7,2.2,19.4,74,OKC,1.55e+07
Dennis Smith\smithde03,455,PG,20,DAL,69,69,29.7,5.9,14.8,0.395,1.5,4.9,0.313,4.3,9.9,0.435,0.446,1.9,2.8,0.694,0.7,3,3.8,5.2,1,0.3,2.8,2.2,15.2,238,DAL,3.81996e+06


In [460]:
#SG
shooting_guard = players.where('Pos', are.equal_to('SG'))
shooting_guard

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Alec Burks\burksal01,78,SG,26,UTA,64,1,16.5,2.7,6.5,0.411,0.7,2.2,0.331,1.9,4.3,0.452,0.467,1.7,1.9,0.863,0.3,2.7,3,1,0.6,0.1,0.9,1.2,7.7,120,UTA,1.15365e+07
Allen Crabbe\crabbal01,114,SG,25,BRK,75,68,29.3,4.5,11,0.407,2.7,7.1,0.378,1.8,3.9,0.461,0.529,1.5,1.8,0.852,0.4,3.9,4.3,1.6,0.6,0.5,1,2.2,13.2,57,BRK,1.85e+07
Andre Roberson\roberan03,429,SG,26,OKC,39,39,26.6,2.2,4.2,0.537,0.2,0.9,0.222,2,3.2,0.627,0.562,0.3,1,0.316,1.9,2.8,4.7,1.2,1.2,0.9,0.8,2.3,5,133,OKC,1e+07
Antonio Blakeney\blakean01,51,SG,21,CHI,19,0,16.5,2.8,7.5,0.371,0.8,2.7,0.288,2,4.8,0.418,0.423,1.6,2.1,0.769,0.2,1.5,1.7,1.1,0.4,0.1,0.6,1,7.9,440,CHI,1.34938e+06
Austin Rivers\riverau01,428,SG,25,LAC,61,59,33.7,5.6,13.2,0.424,2.2,5.9,0.378,3.4,7.3,0.461,0.508,1.7,2.6,0.642,0.4,1.9,2.4,4,1.2,0.3,1.8,2.5,15.1,102,WAS,1.265e+07
Avery Bradley\bradlav01,61,SG,27,TOT,46,46,31.2,5.6,13.6,0.414,1.6,4.4,0.369,4,9.2,0.436,0.474,1.4,1.8,0.768,0.5,2,2.5,2,1.1,0.2,2.2,2,14.3,112,LAC,1.2e+07
Ben McLemore\mclembe01,331,SG,24,MEM,56,17,19.5,2.8,6.7,0.421,1,2.8,0.346,1.8,3.8,0.477,0.495,0.9,1.1,0.828,0.6,1.9,2.5,0.9,0.7,0.3,1.1,2.5,7.5,199,SAC,5.46e+06
Bogdan Bogdanovic\bogdabo01,54,SG,25,SAC,78,53,27.9,4.4,9.9,0.446,1.7,4.2,0.392,2.8,5.7,0.485,0.529,1.3,1.5,0.84,0.4,2.5,2.9,3.3,0.9,0.2,1.6,2.2,11.8,142,SAC,9e+06
Bradley Beal\bealbr01,37,SG,24,WAS,82,82,36.3,8.3,18.1,0.46,2.4,6.5,0.375,5.9,11.6,0.507,0.527,3.6,4.5,0.791,0.7,3.7,4.4,4.5,1.2,0.4,2.6,2,22.6,23,WAS,2.54343e+07
CJ McCollum\mccolcj01,322,SG,26,POR,81,81,36.1,8.2,18.6,0.443,2.3,5.9,0.397,5.9,12.7,0.465,0.506,2.6,3.1,0.836,0.7,3.3,4,3.4,1,0.4,1.9,2.1,21.4,19,POR,2.57598e+07


In [461]:
#SF
small_forward = players.where('Pos', are.equal_to('SF'))
small_forward

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Abdel Nader\naderab01,363,SF,24,BOS,48,1,10.9,1,3.1,0.336,0.5,1.4,0.354,0.6,1.8,0.321,0.413,0.5,0.8,0.59,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3,422,OKC,1.37824e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,383,GSW,1.34938e+06
Alfonzo McKinnie\mckinal01,330,SF,25,TOR,14,0,3.8,0.6,1.1,0.533,0.2,0.6,0.333,0.4,0.4,0.833,0.633,0.1,0.2,0.667,0.1,0.4,0.5,0.1,0.1,0.1,0.2,0.6,1.5,446,GSW,1.34938e+06
Andre Iguodala\iguodan01,234,SF,34,GSW,64,7,25.3,2.3,5,0.463,0.5,1.8,0.282,1.8,3.2,0.567,0.514,0.9,1.4,0.632,0.8,3.1,3.8,3.3,0.8,0.6,1,1.5,6,71,GSW,1.6e+07
Andrew Wiggins\wiggian01,514,SF,22,MIN,82,82,36.3,6.9,15.9,0.438,1.4,4.1,0.331,5.6,11.7,0.475,0.481,2.5,3.8,0.643,1,3.4,4.4,2,1.1,0.6,1.7,2,17.7,21,MIN,2.54672e+07
Bojan Bogdanovic\bogdabo02,55,SF,28,IND,80,80,30.8,5.1,10.7,0.474,1.9,4.8,0.402,3.1,5.8,0.534,0.565,2.2,2.6,0.868,0.4,3,3.4,1.5,0.7,0.1,1.3,1.6,14.3,129,IND,1.05e+07
C.J. Miles\milescj01,337,SF,30,TOR,70,3,19.1,3.2,8.6,0.379,2.3,6.5,0.361,0.9,2.1,0.434,0.516,1.2,1.4,0.835,0.3,1.9,2.2,0.8,0.5,0.3,0.6,1.9,10,155,TOR,8.33333e+06
Cedi Osman\osmande01,385,SF,22,CLE,61,12,11,1.5,3,0.484,0.5,1.4,0.368,1,1.6,0.586,0.57,0.4,0.8,0.565,0.3,1.7,2,0.7,0.4,0,0.5,1.3,3.9,278,CLE,2.775e+06
Chandler Parsons\parsoch01,392,SF,29,MEM,36,8,19.2,3,6.5,0.462,1.4,3.4,0.421,1.6,3.1,0.504,0.571,0.5,0.8,0.63,0.4,2.1,2.5,1.9,0.5,0.3,1,1.5,7.9,32,MEM,2.41073e+07
Danilo Gallinari\gallida01,168,SF,29,LAC,21,21,32,4.9,12.2,0.398,1.7,5.3,0.324,3.1,6.9,0.455,0.469,3.9,4.1,0.931,0.4,4.4,4.8,2,0.6,0.5,1.2,1.4,15.3,41,LAC,2.15876e+07


In [462]:
#PF
power_forward = players.where('Pos', are.equal_to('PF'))
power_forward

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Aaron Gordon\gordoaa01,179,PF,22,ORL,58,57,32.9,6.5,14.9,0.434,2,5.9,0.336,4.5,9,0.497,0.5,2.7,3.9,0.698,1.5,6.4,7.9,2.3,1,0.8,1.8,1.9,17.6,40,ORL,2.15909e+07
Al-Farouq Aminu\aminual01,11,PF,27,POR,69,67,30,3.3,8.4,0.395,1.8,4.9,0.369,1.5,3.5,0.432,0.503,0.9,1.2,0.738,1.4,6.2,7.6,1.2,1.1,0.6,1.1,2,9.3,179,POR,6.95710e+06
Anthony Davis\davisan02,122,PF,24,NOP,75,75,36.4,10.4,19.5,0.534,0.7,2.2,0.34,9.7,17.3,0.558,0.552,6.6,8,0.828,2.5,8.6,11.1,2.3,1.5,2.6,2.2,2.1,28.1,24,NOP,2.54343e+07
Anthony Tolliver\tollian01,482,PF,32,DET,79,14,22.2,2.8,6,0.464,2,4.6,0.436,0.8,1.4,0.561,0.632,1.3,1.7,0.797,0.6,2.4,3.1,1.1,0.4,0.3,0.7,1.8,8.9,195,MIN,5.75e+06
Ben Moore\moorebe01,348,PF,22,IND,2,0,4.5,0,0,nan,0,0,nan,0,0,nan,nan,0,0,nan,0,0.5,0.5,0.5,0,0,0,2,0,445,IND,1.34938e+06
Blake Griffin\griffbl01,191,PF,28,TOT,58,58,34,7.5,17.2,0.438,1.9,5.6,0.345,5.6,11.6,0.482,0.493,4.5,5.7,0.785,1.3,6.1,7.4,5.8,0.7,0.3,2.8,2.4,21.4,5,DET,3.20889e+07
Caleb Swanigan\swanica01,467,PF,20,POR,27,3,7,0.9,2.2,0.4,0,0.3,0.125,0.9,1.9,0.442,0.408,0.4,0.7,0.667,0.6,1.4,2,0.5,0.2,0.1,0.7,1.4,2.3,358,POR,1.74e+06
Carmelo Anthony\anthoca01,17,PF,33,OKC,78,78,32.1,6.1,15,0.404,2.2,6.1,0.357,3.9,8.9,0.437,0.476,1.9,2.5,0.767,0.9,4.9,5.8,1.3,0.6,0.6,1.3,2.5,16.2,20,ATL,2.55343e+07
Carmelo Anthony\anthoca01,17,PF,33,OKC,78,78,32.1,6.1,15,0.404,2.2,6.1,0.357,3.9,8.9,0.437,0.476,1.9,2.5,0.767,0.9,4.9,5.8,1.3,0.6,0.6,1.3,2.5,16.2,308,HOU,2.39389e+06
D.J. Wilson\wilsodj01,524,PF,21,MIL,22,0,3.2,0.4,0.7,0.563,0.1,0.2,0.4,0.3,0.5,0.636,0.625,0,0.1,0.5,0.1,0.4,0.5,0.1,0.1,0,0.2,0.3,1,287,MIL,2.53428e+06


In [463]:
#C
center = players.where('Pos', are.equal_to('C'))
center

Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PS/G,Rk_2,Tm_2,2018-19
Al Horford\horfoal01,227,C,31,BOS,72,72,31.6,5.1,10.5,0.489,1.3,3.1,0.429,3.8,7.3,0.514,0.553,1.3,1.7,0.783,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,13,BOS,2.89287e+07
Al Jefferson\jeffeal01,250,C,33,IND,36,1,13.4,3.1,5.8,0.534,0,0.1,0,3.1,5.7,0.541,0.534,0.8,1,0.833,0.8,3.1,4,0.8,0.4,0.6,0.6,1.8,7,235,IND,4e+06
Alex Len\lenal01,292,C,24,PHO,69,13,20.2,3.1,5.5,0.566,0,0,0.333,3.1,5.4,0.568,0.567,2.3,3.3,0.684,2.5,5,7.5,1.2,0.4,0.9,1.1,2.3,8.5,227,ATL,4.35e+06
Amir Johnson\johnsam01,254,C,30,PHI,74,18,15.8,1.9,3.5,0.538,0.1,0.4,0.313,1.8,3.1,0.57,0.558,0.7,1.1,0.612,1.7,2.8,4.5,1.6,0.6,0.6,0.7,2.6,4.6,301,PHI,2.39389e+06
Andre Drummond\drumman01,140,C,24,DET,78,78,33.7,6,11.3,0.529,0,0.1,0,6,11.2,0.536,0.529,3.1,5.1,0.605,5.1,10.9,16,3,1.5,1.6,2.6,3.2,15,25,DET,2.54343e+07
Ante Zizic\zizican01,539,C,21,CLE,32,2,6.7,1.5,2.1,0.731,0,0,nan,1.5,2.1,0.731,0.731,0.7,0.9,0.724,0.8,1.1,1.9,0.2,0.1,0.4,0.3,0.9,3.7,342,CLE,1.95276e+06
Aron Baynes\baynear01,35,C,31,BOS,81,67,18.3,2.6,5.5,0.471,0,0.3,0.143,2.6,5.2,0.487,0.474,0.7,1,0.756,1.6,3.8,5.4,1.1,0.3,0.6,1,2.5,6,210,BOS,5.1936e+06
Bam Adebayo\adebaba01,4,C,20,MIA,69,19,19.8,2.5,4.9,0.512,0,0.1,0,2.5,4.8,0.523,0.512,1.9,2.6,0.721,1.7,3.8,5.5,1.5,0.5,0.6,1,2,6.9,271,MIA,2.95584e+06
Bismack Biyombo\biyombi01,48,C,25,ORL,82,25,18.2,2.2,4.3,0.52,0,0,0,2.2,4.3,0.521,0.52,1.2,1.9,0.65,1.8,4,5.7,0.8,0.3,1.2,1,1.9,5.7,66,CHO,1.7e+07
Brook Lopez\lopezbr01,302,C,29,LAL,74,72,23.4,5,10.7,0.465,1.5,4.4,0.345,3.5,6.3,0.549,0.536,1.5,2.1,0.703,1,3,4,1.7,0.4,1.3,1.3,2.6,13,253,MIL,3.382e+06


### Data Table Processing Function and Table Creations###

 Function that creates a table of statistic sum per position, including salary average for that position:

In [464]:
# input table of players for each position, outputs table with sums of each category of statistic:
def stat_sums_table(table):
    fg_sum = np.sum(table.column('FG'))
    fga_sum = np.sum(table.column('FGA'))
    fgpct_sum = np.sum(table.column('FG%'))
    three_p_sum = np.sum(table.column('3P'))
    three_pa_sum = np.sum(table.column('3PA'))
    three_ppct_sum = np.sum(table.column('3P%'))
    two_p_sum = np.sum(table.column('2P'))
    two_pa_sum = np.sum(table.column('2PA'))
    two_ppct_sum = np.sum(table.column('2P%'))
    efgpct_sum = np.sum(table.column('eFG%'))
    orb_sum = np.sum(table.column('ORB'))
    drb_sum = np.sum(table.column('DRB'))
    trb_sum = np.sum(table.column('TRB'))
    ast_sum = np.sum(table.column('AST'))
    stl_sum = np.sum(table.column('STL'))
    blk_sum = np.sum(table.column('BLK'))
    tov_sum = np.sum(table.column('TOV'))
    pf_sum = np.sum(table.column('PF'))
    pts_sum = np.sum(table.column('PS/G'))
    salary_sum = np.sum(table.column('2018-19'))

    stat_cats = ('FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'ORB', 'DRB', 
                    'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PPG', '2018-19 Salary')

    stat_sums = (fg_sum, fga_sum, fgpct_sum, three_p_sum, three_pa_sum, three_ppct_sum,
                  two_p_sum, two_pa_sum, two_ppct_sum, efgpct_sum, orb_sum, drb_sum,
                  trb_sum, ast_sum, stl_sum, blk_sum, tov_sum, pf_sum, pts_sum, salary_sum)
    return Table().with_columns('Category', stat_cats, 'Sums', stat_sums)

In [465]:
point_guard_sums = stat_sums_table(point_guard)
point_guard_sums.show()

Category,Sums
FG,240.8
FGA,547.7
FG%,25.031
3P,67.6
3PA,188
3P%,18.755
2P,173
2PA,360
2P%,27.721
eFG%,28.478


In [466]:
shooting_guard_sums = stat_sums_table(shooting_guard)
power_forward_sums = stat_sums_table(power_forward)
small_forward_sums = stat_sums_table(small_forward)
center_sums = stat_sums_table(center)

In [467]:
shooting_guard_sums = shooting_guard_sums.sort('Sums', descending=True)
power_forward_sums = power_forward_sums.sort('Sums', descending=True)
small_forward_sums = small_forward_sums.sort('Sums', descending=True)
center_sums = center_sums.sort('Sums', descending=True)

**Tables of statistic averages per position, including salary average for that position:**

In [468]:
# input table of players for each position, outputs table with sums of each category of statistic:
def stat_means_table(table):
    fg = np.mean(table.column('FG'))
    fga = np.mean(table.column('FGA'))
    fgpct = np.mean(table.column('FG%'))
    three_p = np.mean(table.column('3P'))
    three_pa = np.mean(table.column('3PA'))
    three_ppct = np.mean(table.column('3P%'))
    two_p = np.mean(table.column('2P'))
    two_pa = np.mean(table.column('2PA'))
    two_ppct = np.mean(table.column('2P%'))
    efgpct = np.mean(table.column('eFG%'))
    orb = np.mean(table.column('ORB'))
    drb = np.mean(table.column('DRB'))
    trb = np.mean(table.column('TRB'))
    ast = np.mean(table.column('AST'))
    stl = np.mean(table.column('STL'))
    blk = np.mean(table.column('BLK'))
    tov = np.mean(table.column('TOV'))
    pf = np.mean(table.column('PF'))
    pts = np.mean(table.column('PS/G'))
    salary = np.mean(table.column('2018-19'))

    stat_cats = ('FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'ORB', 'DRB', 
                    'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PPG', '2018-19 Salary')

    stat_averages = (fg, fga, fgpct, three_p, three_pa, three_ppct,
                  two_p, two_pa, two_ppct, efgpct, orb, drb,
                  trb, ast, stl, blk, tov, pf, pts, salary)
    return Table().with_columns('Category', stat_cats, 'Means', stat_averages)

In [469]:
point_guard_means = stat_means_table(point_guard)
point_guard_means.show()

Category,Means
FG,4.22456
FGA,9.60877
FG%,0.43914
3P,1.18596
3PA,3.29825
3P%,0.329035
2P,3.03509
2PA,6.31579
2P%,0.486333
eFG%,0.499614


In [470]:
shooting_guard_means = stat_means_table(shooting_guard)
power_forward_means = stat_means_table(power_forward)
small_forward_means = stat_means_table(small_forward)
center_means = stat_means_table(center)

In [471]:
shooting_guard_means = shooting_guard_means.sort('Means', descending=True)
power_forward_means = power_forward_means.sort('Means', descending=True)
small_forward_means = small_forward_means.sort('Means', descending=True)
center_means = center_means.sort('Means', descending=True)

### Analyze the rank of the positions' means per category. 

In [472]:
# change the 'means' column title to 'position means'
def change_title(table, position):
    table = table.relabel('Means', position + ' Means')

In [473]:
# join means columns from all tables into one to compare:
all_position_means = point_guard_means.join('Category', shooting_guard_means).join('Category', power_forward_means).join('Category', small_forward_means).join('Category', center_means)
all_position_means

Category,Means,Means_2,Means_3,Means_4,Means_5
2018-19 Salary,1.00296e+07,9.20274e+06,1.04822e+07,9.80793e+06,9.92559e+06
2P,3.03509,2.61622,2.88475,2.19804,3.00286
2P%,0.486333,0.476459,nan,0.520314,0.547214
2PA,6.31579,5.42027,5.52542,4.32157,5.40429
3P,1.18596,1.52838,1.02373,1.04902,0.268571
3P%,0.329035,0.356311,nan,nan,nan
3PA,3.29825,4.15,2.80678,2.92353,0.801429
AST,4.38596,2.36216,1.65763,1.48824,1.29286
BLK,0.235088,0.293243,0.535593,0.335294,0.767143
DRB,2.53684,2.46622,3.80678,2.63922,4.18429


In [348]:
all_position_means.show()

Category,Means,Means_2,Means_3,Means_4,Means_5
2018-19 Salary,1.20346e+07,1.11578e+07,1.30362e+07,1.2044e+07,1.15952e+07
2P,3.21591,2.94386,3.32444,2.35641,3.33793
2P%,0.482318,0.482667,0.507067,0.508821,0.5475
2PA,6.64545,6.04211,6.35778,4.63333,6.00862
3P,1.33636,1.74737,1.15778,1.18205,0.306897
3P%,0.343386,0.364526,nan,nan,nan
3PA,3.64545,4.71404,3.15333,3.25128,0.901724
AST,4.61818,2.65263,1.95111,1.66923,1.43103
BLK,0.218182,0.335088,0.577778,0.361538,0.827586
DRB,2.53864,2.67193,4.24889,2.80769,4.67414


In [349]:
point_guard.to_csv('point_guard.csv')
shooting_guard.to_csv('shooting_guard_guard.csv')
power_forward.to_csv('power_forward.csv')
center.to_csv('center.csv')
small_forward.to_csv('small_forward.csv')

### Using pandas to Generate DataFrames ###

In [474]:
#Creating Pandas type dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
df_pg = pd.read_csv('point_guard.csv')
f_pg = df_pg.isnull().values.any()

In [475]:
df_pg = pd.read_csv('point_guard.csv')
df_sg = pd.read_csv('shooting_guard_guard.csv')
df_pf = pd.read_csv('power_forward.csv')
df_c = pd.read_csv('center.csv')
df_sf = pd.read_csv('small_forward.csv')



In [358]:
frames = [df_pg, df_sg]
df_g = pd.concat(frames)
# Join PG and SG

In [359]:
frames = [df_sf, df_pf]
df_f = pd.concat(frames)

#Joining Forwards

In [479]:
df_f.head()

,Player,Rk_x,Pos,Age,Tm,G,GS,MP,FG,FGA,...,AST%,STL%,BLK%,TOV%,USG%,WS,WS/48,BPM,VORP,zscore_PPG
0,Andre Iguodala\iguodan01,234,SF,34.0,GSW,64.0,7.0,25.3,2.3,5.0,...,16.2,1.6,1.9,15.8,11.5,3.2,0.094,0.2,0.9,-0.787122
1,Andrew Wiggins\wiggian01,514,SF,22.0,MIN,82.0,82.0,36.3,6.9,15.9,...,8.2,1.5,1.5,8.8,23.4,2.1,0.034,-2.5,-0.4,1.098747
2,Bojan Bogdanovic\bogdabo02,55,SF,28.0,IND,80.0,80.0,30.8,5.1,10.7,...,7.1,1.1,0.3,10.1,19.0,5.4,0.105,-0.7,0.8,0.550717
3,C.J. Miles\milescj01,337,SF,30.0,TOR,70.0,3.0,19.1,3.2,8.6,...,6.0,1.4,1.3,5.7,22.3,2.9,0.103,-0.5,0.5,-0.142380
4,Chandler Parsons\parsoch01,392,SF,29.0,MEM,36.0,8.0,19.2,3.0,6.5,...,16.4,1.3,1.5,12.5,18.3,1.1,0.077,-1.2,0.1,-0.480870


### Z-Scores for Specific Statistics ###

Z-scores are used to standarize categories that are represented in percentages because the decimal differences will give these categories a larger weight if used in its raw state. 

In [480]:
df_g['zscore_FG%'] = ((df_g['FG%'] - df_g['FG%'].mean()) /df_g['FG%'].std())
df_f['zscore_FG%'] = ((df_f['FG%'] - df_f['FG%'].mean()) /df_f['FG%'].std())
df_c['zscore_FG%'] = ((df_c['FG%'] - df_c['FG%'].mean()) /df_c['FG%'].std())
#Creating Z score for FG%

In [481]:
df_g['zscore_3P%'] = ((df_g['3P%'] - df_g['3P%'].mean()) /df_g['3P%'].std())
df_f['zscore_3P%'] = ((df_f['3P%'] - df_f['3P%'].mean()) /df_f['3P%'].std())
df_c['zscore_3P%'] = ((df_c['3P%'] - df_c['3P%'].mean()) /df_c['3P%'].std())
#Creating Z score for 3P%

In [482]:
df_g['zscore_PPG'] = ((df_g['PS/G'] - df_g['PS/G'].mean()) /df_g['PS/G'].std())
df_f['zscore_PPG'] = ((df_f['PS/G'] - df_f['PS/G'].mean()) /df_f['PS/G'].std())
df_c['zscore_PPG'] = ((df_c['PS/G'] - df_c['PS/G'].mean()) /df_c['PS/G'].std())

#Creating Z score for PPS

In [484]:
df_adv = pd.read_csv('rookies_2017-18_Advanced.csv')

#import advanced stats

In [485]:
df_adv.head()

,Rk,Player,PER,TS%,3PAr,FTr,AST%,STL%,BLK%,TOV%,USG%,WS,WS/48,BPM,VORP
0,1,Alex Abrines\abrinal01,9.0,0.567,0.759,0.158,3.4,1.7,0.6,7.4,12.7,2.2,0.094,-2.2,-0.1
1,2,Quincy Acy\acyqu01,8.2,0.525,0.800,0.164,6.0,1.2,1.6,13.3,14.4,1.0,0.036,-2.2,-0.1
2,3,Steven Adams\adamsst01,20.6,0.630,0.003,0.402,5.5,1.8,2.8,13.2,16.7,9.7,0.187,3.3,3.3
3,4,Bam Adebayo\adebaba01,15.7,0.570,0.021,0.526,11.0,1.2,2.5,13.6,15.9,4.2,0.148,0.2,0.8
4,5,Arron Afflalo\afflaar01,5.8,0.516,0.432,0.160,6.2,0.3,1.1,10.8,12.5,0.1,0.009,-5.8,-0.7


In [489]:
df_g.head()

,Player,Rk_x,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FTr_y,AST%_y,STL%_y,BLK%_y,TOV%_y,USG%_y,WS_y,WS/48_y,BPM_y,VORP_y
0,Chris Paul\paulch01,396,PG,32.0,HOU,58.0,58.0,31.8,6.3,13.8,...,0.277,40.9,2.6,0.6,12.5,24.5,10.2,0.265,7.1,4.3
1,Cory Joseph\josepco01,269,PG,26.0,IND,82.0,17.0,27.0,3.1,7.4,...,0.156,16.1,1.8,0.7,11.9,14.8,3.1,0.068,-1.2,0.5
2,D.J. Augustin\augusdj01,24,PG,30.0,ORL,75.0,36.0,23.5,3.3,7.2,...,0.350,24.4,1.5,0.0,16.5,18.6,4.1,0.110,-0.9,0.5
3,Damian Lillard\lillada01,298,PG,27.0,POR,73.0,73.0,36.6,8.5,19.4,...,0.380,30.9,1.4,0.8,11.1,30.6,12.6,0.227,6.7,5.9
4,Dante Exum\exumda01,153,PG,22.0,UTA,14.0,0.0,16.8,3.0,6.2,...,0.356,29.7,1.7,1.1,16.6,23.1,0.7,0.133,-1.2,0.0


In [490]:
#Merge the advanced stats

df_g = pd.merge(df_g, df_adv, left_on ='Player', right_on = 'Player', how = 'left')
df_f = pd.merge(df_f, df_adv, left_on ='Player', right_on = 'Player', how = 'left')
df_c = pd.merge(df_c, df_adv, left_on ='Player', right_on = 'Player', how = 'left')

In [491]:
df_c.head()

,Player,Rk_x,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FTr_y,AST%_y,STL%_y,BLK%_y,TOV%_y,USG%_y,WS_y,WS/48_y,BPM_y,VORP_y
0,Al Horford\horfoal01,227,C,31.0,BOS,72.0,72.0,31.6,5.1,10.5,...,0.159,23.6,0.9,2.9,14.1,18.4,7.8,0.165,4.0,3.5
1,Al Jefferson\jeffeal01,250,C,33.0,IND,36.0,1.0,13.4,3.1,5.8,...,0.173,10.0,1.7,4.2,8.6,22.5,1.6,0.158,-0.5,0.2
2,Alex Len\lenal01,292,C,24.0,PHO,69.0,13.0,20.2,3.1,5.5,...,0.611,9.1,0.9,3.5,14.0,17.0,4.5,0.154,0.8,1.0
3,Amir Johnson\johnsam01,254,C,30.0,PHI,74.0,18.0,15.8,1.9,3.5,...,0.327,13.9,1.8,3.1,15.6,12.8,4.1,0.169,2.9,1.5
4,Andre Drummond\drumman01,140,C,24.0,DET,78.0,78.0,33.7,6.0,11.3,...,0.448,14.3,2.2,4.3,15.9,21.2,10.3,0.189,5.5,5.0


In [492]:
from sklearn.model_selection import train_test_split
lm = linear_model.LinearRegression()

In [493]:
list(df_g.columns.values)

['Player',
 'Rk_x',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PS/G',
 'Rk_2',
 'Tm_2',
 '2018-19',
 'zscore_FG%',
 'zscore_3P%',
 'Rk_y',
 'PER_x',
 'TS%_x',
 '3PAr_x',
 'FTr_x',
 'AST%_x',
 'STL%_x',
 'BLK%_x',
 'TOV%_x',
 'USG%_x',
 'WS_x',
 'WS/48_x',
 'BPM_x',
 'VORP_x',
 'zscore_PPG',
 'Rk_x',
 'PER_y',
 'TS%_y',
 '3PAr_y',
 'FTr_y',
 'AST%_y',
 'STL%_y',
 'BLK%_y',
 'TOV%_y',
 'USG%_y',
 'WS_y',
 'WS/48_y',
 'BPM_y',
 'VORP_y',
 'Rk_y',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'WS',
 'WS/48',
 'BPM',
 'VORP']

## Linear Regression ##

### Linear Regression for Guards ###

In [494]:
#Perform linear regression for each positions

f= ['3P','AST','STL','PS/G','BPM','TOV']
X = df_g[f]
Y= df_g['2018-19']
model = lm.fit(X,Y)
print(model.coef_)
print(model.intercept_)

[  432038.34220447   886452.77351222  2140116.20723071   776926.27393328
   275649.11015892 -1300075.73396038]
-2012446.7069338765


The formula for Guard salary is given by:

***Guard Salary*** = 0.44 * ***3P*** + 0.89 * ***AST*** + 2.1 * ***STL*** + 0.77 * ***PPG*** + .28 * ***BPM*** - 1.3 * ***TOV***- 2.1

### Linear Regression for Forwards ###

In [495]:
f= ['zscore_PPG','DRB','STL','BLK','PF']
X = df_f[f]
Y= df_f['2018-19']
model = lm.fit(X,Y)
print(model.coef_)
print(model.intercept_)

[ 5312749.11864885    60102.72534906  1005830.53383841  2062096.15767864
 -1614964.91614739]
13607730.432828384


The formula for Forward salary is given by:

***Forward Salary*** = 5.3 * ***z_score_PPG*** + 0.60 * ***DRB*** + 1.0 * ***STL*** + 2.1 * ***BLK*** - 1.6 * ***PF*** + 13.6

### Linear Regression for Centers ###

In [496]:
f= ['PS/G','TRB','BLK','FG%','PF']
X = df_c[f]
Y= df_c['2018-19']
model = lm.fit(X,Y)
print(model.coef_)
print(model.intercept_)

[  401417.15508358  1311366.468154    1478763.74325252 19071247.63496706
 -6106470.60691408]
1068102.4043343589


The formula for Center salary is given by:

***Center Salary*** = 0.4 * ***PPG*** + 1.3 * ***TRB*** + 1.5 * ***BLK*** + 19.1 * ***FG%*** - 6.1 * ***PF*** + 1.07

# Final Results #

The results of the linear regression were formulas that are used to find a contract value for a player in the NBA based on his statistics for that category. Since there are three major positions in the NBA (guard, forward, and center), there are three formulas: 

## Guard Contract Value Model ##

***Guard Salary*** = 0.44 * ***3P*** + 0.89 * ***AST*** + 2.1 * ***STL*** + 0.77 * ***PPG*** + .28 * ***BPM*** - 1.3 * ***TOV***- 2.1

## Forward Contract Value Model ##

***Forward Salary*** = 5.3 * ***z_score_PPG*** + 0.60 * ***DRB*** + 1.0 * ***STL*** + 2.1 * ***BLK*** - 1.6 * ***PF*** + 13.6

## Center Contract Value Model ##

***Center Salary*** = 0.4 * ***PPG*** + 1.3 * ***TRB*** + 1.5 * ***BLK*** + 19.1 * ***FG%*** - 6.1 * ***PF*** + 1.07